# 动态路由

Bridgic 支持根据运行时条件动态路由到相应的 Worker。现在让我们通过一个示例来理解它。

<br>
<div style="text-align: center;">
<img src="../../../imgs/dynamic_router.png" alt="参数传递" width="400" height="250">
</div>
<br>

## 领域聊天机器人

动态路由是聊天机器人中的一个常见需求。例如，它涉及分析输入查询以确定其主要领域或领域，然后将其动态路由到相应的处理逻辑。

用户输入原始查询，我们识别它是与医疗相关还是与法律相关的问题，然后用专业逻辑回答它。这个过程分为三个步骤：

1. 接收用户输入
2. 领域识别和动态路由到正确的 Worker
3. 回答查询

### 1. 初始化

在我们开始之前，让我们准备运行环境。

In [ ]:
# Get the environment variables.
import os
from typing import Literal

_api_key = os.environ.get("OPENAI_API_KEY")
_api_base = os.environ.get("OPENAI_API_BASE")
_model_name = os.environ.get("OPENAI_MODEL_NAME")

# Import the necessary packages.
from pydantic import BaseModel
from bridgic.core.model.protocols import PydanticModel
from bridgic.core.automa import GraphAutoma, worker
from bridgic.core.model.protocols import Choice
from bridgic.core.model.types import Message, Role
from bridgic.llms.openai import OpenAILlm, OpenAIConfiguration

llm = OpenAILlm(  # the llm instance
    api_base=_api_base,
    api_key=_api_key,
    configuration=OpenAIConfiguration(model=_model_name),
    timeout=20,
)

### 2. 完整

我们假设收到的用户查询是一个字符串。让我们实现域聊天机器人来回答用户查询。

In [ ]:
class Domain(BaseModel):
    choices: Literal["medical", "legal"]

In [ ]:
class DomainChatbot(GraphAutoma):
    @worker(is_start=True)
    async def pre_query(self, query: str):  # receive user input and preprocess it
        return query

    @worker(dependencies=["pre_query"])
    async def domain_recognition(self, query: str):  # domain recognition and dynamic routing
        response: Domain = await llm.astructured_output(
            constraint=PydanticModel(model=Domain),
            messages=[
                Message.from_text(text="Please identify the domain of the following query", role=Role.SYSTEM),
                Message.from_text(text=query, role=Role.USER),
            ]
        )
        print(f'The query domain is: {response}')  # print the response
        
        # dynamic routing
        if response.choices == "medical":
            return self.ferry_to('answer_medical_query', query)
        elif response.choices == "legal":
            return self.ferry_to('answer_legal_query', query)
    
    @worker(is_output=True)
    async def answer_medical_query(self, query: str):  # answer medical-related query
        response = await llm.achat(
            messages=[
                Message.from_text(text="You are a medical expert. Answer the following medical-related query", role=Role.SYSTEM),
                Message.from_text(text=query, role=Role.USER),
            ]
        )
        return response.message.content

    @worker(is_output=True)
    async def answer_legal_query(self, query: str):  # answer legal-related query
        response = await llm.achat(
            messages=[
                Message.from_text(text="You are a legal expert. Answer the following legal-related query", role=Role.SYSTEM),
                Message.from_text(text=query, role=Role.USER),
            ]
        )
        return response.message.content


让我们运行它！

In [5]:
query = "What is the aspirin?"
domain_chatbot = DomainChatbot()
await domain_chatbot.arun(query)

The query domain is: medical


'Aspirin, also known as acetylsalicylic acid, is a widely used medication with several important medical properties. It is one of the oldest and most well-known over-the-counter (OTC) drugs, originally derived from willow bark.\n\n### Key Uses of Aspirin:\n\n1. **Pain Relief (Analgesic)**  \n   Aspirin helps relieve mild to moderate pain such as headaches, toothaches, muscle aches, and menstrual cramps.\n\n2. **Reducing Fever (Antipyretic)**  \n   It lowers body temperature in cases of fever, such as in infections.\n\n3. **Reducing Inflammation (Anti-inflammatory)**  \n   Aspirin helps reduce inflammation, which can be beneficial in conditions like rheumatoid arthritis or other inflammatory disorders.\n\n4. **Preventing Blood Clots (Antiplatelet Effect)**  \n   This is one of its most important therapeutic uses. Aspirin inhibits the formation of blood clots by blocking the action of a substance called cyclooxygenase (COX), which reduces the production of thromboxane—a molecule that pro

> 如果在执行上述过程时遇到 Readtimeout 错误，可能是因为在初始化 llm 时设置的超时时间过短，导致模型在完成响应之前超时。

太好了！我们已经成功完成了领域聊天机器人。

<div style="text-align: center; margin: 2rem 0;">
<hr style="border: none; border-top: 2px solid #e2e8f0;">
</div>

## 我们学到了什么？

我们已经使用 Bridgic 实现了路由机制。

### Ferry To

在 Bridgic 中，我们使用 [`ferry_to`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.ferry_to) 机制来实现动态路由。它的第一个参数是将要被路由指向的 worker 的 `key`，第二个参数是传递给目标 worker 的输入参数。